<center><b> 1D time-dependent Frantz Nodvik equations </b></center>

<br> du1/dt + gamma*K*u1*u2 = 0           u1 = Delta = # density of excited states 
<br> du2/dt + c*du2/dx - K*u1*u2 = 0      u2 = n_ph  = # density of photons
<br>                                      c = speed of light
<br>                                      K = sigma * c
<br> IC:  TBD...
<br> BC:  TBD...
<br> This code is distributed under the Apache License, Version 2.0, January 2004.
<br> Copyright (c) 2020 RadiaSoft LLC, All rights reserved
<br> Author:  David Bruhwiler                                     
<br> The laser pulse duration $T_{laser}$ is typically 200 ps to 1 ns.
<br> The Ti-Sapphire phase relaxation time $T_{crystal}$ is between $10 fs$ and $10 ps$.
<br> Because $T_{crystal} << T_{laser}$, we can use the Frantz-Nodvik (F-N) rate equations.
<br> This approach can be used for other crystals as well.

In [1]:
# Primary library for the FEniCS Python API (should not import *)
from dolfin import *
from pykern import pkcollections
import scipy.constants

In [2]:
def frantz_nodvik_1d_solver(fn1d):
    """Frantz-Nodvik 1D solver for wavefront amplification in a pumped crystal amplifier

    Args:
        fn1d (PKDict): derived class of Python dict

    Returns:
        NA
    """
    
    # define the variational problem via function F
    K = Constant(fn1d.K)
    C = Constant(fn1d.c)
    DT = Constant(fn1d.dt)
    G = Constant(fn1d.c_gamma)
    F = ( (fn1d.u_1 - fn1d.u_n1) * fn1d.v_1 / DT \
        + G * K * fn1d.u_1 * fn1d.u_2 * fn1d.v_1 \
        + (fn1d.u_2 - fn1d.u_n2) * fn1d.v_2 / DT \
        + C * fn1d.u_2.dx(0) * fn1d.v_2 \
        - K * fn1d.u_1 * fn1d.u_2 * fn1d.v_2 \
        ) * dx
    
    # solve the system of equations
    solve(F==0, fn1d.u)
    fn1d.u_n.assign(fn1d.u)
    
    return fn1d

In [3]:
def frantz_nodvik_1d_setup():
    """Setup choices for Frantz-Nodvik 1D solver

    Args:
        NA

    Returns:
        fn1d (object):  instance of the DataFrantzNodvik1D class
    """
    # specify the desired parameters (edit by hand for now)
    # create a Pykern Dict object to hold the data
    fn1d = pkcollections.PKDict()
    
    fn1d.sim_time = 1.       # [s]
    fn1d.n_steps = 100
    fn1d.dt = fn1d.sim_time  \
            / fn1d.n_steps      # [s]
    fn1d.n_dx = 8
    fn1d.c_gamma = 1.          # dimensionless
    fn1d.c_sigma = 0.001        # [m^2]
    fn1d.c_width = 1.         # [m]
    fn1d.c_density = 1.      # [m^-3]
    fn1d.lp_density = 1.       # [m^-3]
    fn1d.lp_width = 1.       # [s]
    
#    fn1d.c = scipy.constants.c         # speed of light [m/s]
    fn1d.c = 1.
    fn1d.K = fn1d.c_sigma * fn1d.c     # [m^3/s]
    
    # create a 1D mesh on the interval [0,1]
    fn1d.x_left = 0.
    fn1d.x_right = 1.
    fn1d.mesh = UnitIntervalMesh(fn1d.n_dx)

    # let's try continuous line segments
    fn1d.p1 = FiniteElement('CG','interval',1)
    
    # define an 'element' appropriate for two 1D functions
    fn1d.element = MixedElement([fn1d.p1, fn1d.p1])
    
    # define the test functions: v=[v_1, v_2]
    fn1d.v = FunctionSpace(fn1d.mesh, fn1d.element)
    fn1d.v_1, fn1d.v_2 = TestFunctions(fn1d.v)

    # define solution functions: u=[u_1, u_2]
    fn1d.u = Function(fn1d.v)
    fn1d.u_1, fn1d.u_2 = split(fn1d.u)

    # solutions from the previous time step: u_n=[u_n1, u_n2]
    fn1d.u_n = Function(fn1d.v)
    fn1d.u_n1, fn1d.u_n2 = split(fn1d.u_n)
    
    # specify and apply the initial conditions
    u0 = Expression(('c_density', 'lp_density'), degree=1, c_density=1., lp_density=1.)
    u0.c_density = fn1d.c_density
    u0.lp_density = fn1d.lp_density
    fn1d.u = project(u0, fn1d.v)

    print('')
    print(' In frantz_nodvik_1d_setup() ...')
    
#    u_array = fn1d.u.vector()
#    print(' u_array = ', u_array[:])

    # set the 'previous' value of u
    fn1d.u_n.assign(fn1d.u)

    return fn1d

In [4]:
# initialize the data object
fn1d = frantz_nodvik_1d_setup()


 In frantz_nodvik_1d_setup() ...


In [5]:
import plotly.graph_objects
import dolfin
import platform
import time
import numpy as np

u_array = fn1d.u.vector()
mesh_pts=fn1d.mesh.coordinates()
print('')
print(' u_array = ', u_array[:])
print(' mesh_pts = ', mesh_pts)

xvals = mesh_pts[:]
# uvals = fn1d.u.compute_vertex_values()
# print(' fn1d.u = ', uvals)

# solve the equations for n_steps time steps
n_steps = 1
print('')
print(' Calling solver ', n_steps, ' times --')

for i_loop in range(0, n_steps):
    fn1d = frantz_nodvik_1d_solver(fn1d)
    
indices = []
# get the "facets" and build an array of the indices of their coordinates
for item in dolfin.cpp.mesh.facets(fn1d.mesh):
    indices.append(item.entities(0).tolist())
    
uvals = fn1d.u.compute_vertex_values()
print('')
print(' uvals = ', uvals)

# provide these indices to plotly so it can draw proper surfaces
indices = np.array(indices)
ii = indices[:]

print(time.ctime(time.time()))
print('')
print('frantz_nodvik_1d results:')
print('  Python version: %s' % (platform.python_version()))
print('  FENICS version %s'% (dolfin.__version__))
print('  Solve the time-dependent 1d Frantz-Nodvik equations.' )

# plot some results
import matplotlib.pyplot as plt
plot(fn1d.u_1, title=('Excited states in the Ti-Sapphire crystal'))
plt.grid(True)
plt.show()
plt.close()

plot(fn1d.u_2, title=('Photon density in the laser wavefront'))
plt.grid(False)
plt.show()
plt.close()

# wrap up
print('')
print(time.ctime(time.time()))


 u_array =  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 mesh_pts =  [[0.   ]
 [0.125]
 [0.25 ]
 [0.375]
 [0.5  ]
 [0.625]
 [0.75 ]
 [0.875]
 [1.   ]]

 Calling solver  1  times --


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to successfully call PETSc function 'MatSetValuesLocal'.
*** Reason:  PETSc error code is: 63 (Argument out of range).
*** Where:   This error was encountered inside /home/vagrant/src/radiasoft/codes/fenics-20201111.211147/dolfin/dolfin/la/PETScMatrix.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------
